# Mini Batches
Iterate over a data stream in mini batches

In [1]:
import pandas as pd
from river import datasets, compose, metrics, preprocessing
from torch import nn
from river_torch import classification
from itertools import islice

In [2]:
dataset = datasets.Phishing()

In [3]:
class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features,5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X

def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [4]:
model_pipeline = compose.Pipeline(
    preprocessing.StandardScaler(),
    classification.Classifier(module=MyModule,loss_fn="binary_cross_entropy",optimizer_fn="sgd")
)
model_pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  Classifier (
    module=<class '__main__.MyModule'>
    loss_fn="binary_cross_entropy"
    optimizer_fn=<class 'torch.optim.sgd.SGD'>
    lr=0.001
    device="cpu"
    seed=42
  )
)

In [5]:
for batch in batcher(dataset,5):
    x,y = zip(*batch)
    x = pd.DataFrame(x)
    y = list(y)
    y_pred = model_pipeline.predict_proba_many(X=x)
    model_pipeline = model_pipeline.learn_many(x, y)    # make the model learn

AttributeError: 'NoneType' object has no attribute 'out_features'